In [41]:
# Jupyter notebook needs this or else it will crash
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.
from gremlin_python.driver.protocol import GremlinServerError # Gremlin server error
from gremlin_python.process.traversal import Pop # for Pop.all_ in select(Pop.all_, 'v')

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [42]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [43]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    success = False

    while not success:

        try:

            if connection:
                # Add an edge labelled 'connection' with a start time of <time>
                g.V().has('name', name1).not_( # NEGATE 
                    __.bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
                        __.has('start', P.lte(time)),
                        __.or_(
                            __.hasNot('end'),
                            __.has('end', P.gt(time))
                        )
                    )
                ).V().has('name', name1).as_("a").V().has('name', name2).as_("b").addE('connection').from_("a").to("b").property('start', time).iterate()

            else:
                # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
                g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').hasNot('end').property('end', time).iterate()
        
            success = True

        except GremlinServerError:
            print("Oops! Backend error!")

In [44]:
def add_type(type: str) -> None:
    """
    Add a vertex to the graph with a 'type' property of <type>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    g.addV().property('name', type).iterate()

def set_type(name: str, type: str) -> None:
    """
    Connect the vertex labelled with <name> to a vertex labelled <type> with an edge going into the <type> vertex labelled with "type".

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    g.V().has('name', name).as_("a").V().has('name', type).as_("b").addE("type").from_("a").to("b").iterate()

In [45]:
# This will put it in V-E-V-E-V-...-V form as a list per path.
def find_paths(name1: str, name2: str, avoid_type: str, time: float):
    """
    Given two vertices labelled with <name1> and <name2>, return the paths that connect the vertices by edges that were active at <time> as a list.

    Avoid vertices of type avoid_type.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    return g.V().has('name', name1).repeat(
            __.bothE('connection').and_(
                __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
                __.or_(
                    __.hasNot('end'),           # end time doesn't have to exist 
                    __.has('end', P.gt(time))  # OR end time must be greater than <time>
                )
            ).otherV().not_(__.outE('type').inV().has('name', avoid_type)).simplePath()
        ).until(__.has('name', name2)).path().toList()

In [46]:
import datetime

# Clear vertices
g.V().drop().iterate()
g.E().drop().iterate()

"""
(Temporary) Naming scheme:
 - COR######: Correlator input
 - ANT######: Antenna
 - DPF######: Dual-Polarization Feed
 - BLN######: (Active) Balun
 - RFT######: RFoF transmitter
 - OPF######: Optical Fiber
 - RFR######: RFoF receiver
 - ADC######: Analog-to-Digital converter
"""

# Set up the types
types = ['COR', 'ANT', 'DPF', 'BLN', 'RFT', 'OPF', 'RFR', 'ADC']
for type in types:
    add_type(type)


# Correlator node name
cor = 'COR000000'

# Add a correlator input node
g.addV().property('name', cor).iterate()
set_type(cor, 'COR')

# number of HIRAX dishes
dishes = 512

# Connections at every 3.
max_time = 30
connections = [(i, bool(i % 2)) for i in range(0, max_time // 3 + 1, 3)]


# Add the components and connect them at different times.
for i in range(1, dishes + 1):

    now = datetime.datetime.now()

    # The names of the components to refer to
    ant = f'ANT{str(i).zfill(6)}'
    dpf = f'DPF{str(i).zfill(6)}'
    bln = (f'BLN{str(2 * i - 1).zfill(6)}', f'BLN{str(2 * i).zfill(6)}')
    rft = (f'RFT{str(2 * i - 1).zfill(6)}', f'RFT{str(2 * i).zfill(6)}')
    opf = (f'OPF{str(2 * i - 1).zfill(6)}', f'OPF{str(2 * i).zfill(6)}')
    rfr = (f'RFR{str(2 * i - 1).zfill(6)}', f'RFR{str(2 * i).zfill(6)}')
    adc = (f'ADC{str(2 * i - 1).zfill(6)}', f'ADC{str(2 * i).zfill(6)}')

    g.addV().property('name', ant).iterate()
    g.addV().property('name', dpf).iterate()

    set_type(ant, 'ANT')
    set_type(dpf, 'DPF')

    for ind in (0, 1):
        g.addV().property('name', bln[ind]).iterate()
        g.addV().property('name', rft[ind]).iterate()
        g.addV().property('name', opf[ind]).iterate()
        g.addV().property('name', rfr[ind]).iterate()
        g.addV().property('name', adc[ind]).iterate()

        set_type(bln[ind], 'BLN')
        set_type(rft[ind], 'RFT')
        set_type(opf[ind], 'OPF')
        set_type(rfr[ind], 'RFR')
        set_type(adc[ind], 'ADC')

    for (time, connection) in connections:

        set_connection(name1=ant, name2=dpf, time=time, connection=connection)

        for ind in (0, 1):

            # Pairs of names to connect
            pairs = [(ant, dpf), (dpf, bln[ind]), (bln[ind], rft[ind]), (rft[ind], opf[ind]), (opf[ind], rfr[ind]), (rfr[ind], adc[ind]), (adc[ind], cor)]

            for pair in pairs:
                set_connection(name1=pair[0], name2=pair[1], time=time, connection=connection)

    print(f"Dish {i} done, took {(datetime.datetime.now() - now).total_seconds()}")



# EXPORT THE GRAPH.
# g.io("path_benchmark_small_graph.xml").write().iterate()




Dish 1 done, took 3.165289
Dish 2 done, took 1.860513
Dish 3 done, took 1.674536
Dish 4 done, took 0.935674
Dish 5 done, took 0.865013
Dish 6 done, took 0.857534
Dish 7 done, took 0.861199
Dish 8 done, took 1.023952
Dish 9 done, took 0.993023
Dish 10 done, took 0.998995
Dish 11 done, took 0.940345
Dish 12 done, took 0.8225
Dish 13 done, took 0.881127
Dish 14 done, took 0.821426
Dish 15 done, took 0.831011
Dish 16 done, took 0.899568
Dish 17 done, took 0.897668
Dish 18 done, took 0.894811
Dish 19 done, took 0.916137
Dish 20 done, took 0.906023
Dish 21 done, took 0.950398
Dish 22 done, took 0.899105
Dish 23 done, took 0.910906
Dish 24 done, took 0.977578
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f3a44dc59a0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f3a44de1340>
Exception ignored in: <function ClientResponse.__del__ at 0x7f3a47fdd0d0>
Traceback (most recent call last):
  File "/home/azszavyalov/.local/lib/p

In [47]:
import datetime

# To benchmark, try finding all of the paths from the correlator to each of the antennas, and time it.
iterations = 1

times_to_query = [1]

total = 0

print(f"Dishes: {dishes}")
print(f"Max time: {max_time}")
print(f"Iterations: {iterations}")

for _ in range(iterations):
    for i in range(len(times_to_query)):
        time = times_to_query[i]
        for dish in range(dishes):

            success = False
            while not success:
                try:
                    then = datetime.datetime.now()
                    find_paths(name1=cor, name2=f'ANT{str(dish).zfill(6)}', time=time, avoid_type='')

                    total += (datetime.datetime.now() - then).total_seconds()

                    print(f"Dish {dish} done")

                    success = True
                except GremlinServerError:
                    print("Uh oh! Backend exception!")

        print(f"{len(times_to_query) - i - 1} times left to check!")

print("Total time:", total)
print("Average time per call:", total / (iterations * len(times_to_query) * dishes))
        

Dishes: 128
Max time: 30
Iterations: 1
Dish 0 done
Dish 1 done
Dish 2 done
Dish 3 done
Dish 4 done
Dish 5 done
Dish 6 done
Dish 7 done
Dish 8 done
Dish 9 done
Dish 10 done
Dish 11 done
Dish 12 done
Dish 13 done
Dish 14 done
Dish 15 done
Dish 16 done
Dish 17 done
Dish 18 done
Dish 19 done
Dish 20 done
Dish 21 done
Dish 22 done
Dish 23 done
Dish 24 done
Dish 25 done
Dish 26 done
Dish 27 done
Dish 28 done
Dish 29 done
Dish 30 done
Dish 31 done
Dish 32 done
Dish 33 done
Dish 34 done
Dish 35 done
Dish 36 done
Dish 37 done
Dish 38 done
Dish 39 done
Dish 40 done
Dish 41 done
Dish 42 done
Dish 43 done
Dish 44 done
Dish 45 done
Dish 46 done
Dish 47 done
Dish 48 done
Dish 49 done
Dish 50 done
Dish 51 done
Dish 52 done
Dish 53 done
Dish 54 done
Dish 55 done
Dish 56 done
Dish 57 done
Dish 58 done
Dish 59 done
Dish 60 done
Dish 61 done
Dish 62 done
Dish 63 done
Dish 64 done
Dish 65 done
Dish 66 done
Dish 67 done
Dish 68 done
Dish 69 done
Dish 70 done
Dish 71 done
Dish 72 done
Dish 73 done
Dish 74 d